# SVM

In [50]:
import cv2
import numpy as np
import os
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define the image directory and HOG parameters
img_dir = "C:/WPy64-31110/notebooks/docs/Shoe vs Sandal vs Boot Image Dataset/"
classes = ['boot', 'sandal', 'shoe']
winSize = (64,64)
blockSize = (16,16)
blockStride = (8,8)
cellSize = (8,8)
nbins = 9

# Create the HOG descriptor object
hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins)

# Create empty lists to store the features and labels
X = []
y = []

# Loop through each image in the directory and extract features
for class_name in classes:
    class_dir = os.path.join(img_dir, class_name)
    for filename in os.listdir(class_dir):
        if filename.endswith((".jpg", ".jpeg")):
            # Load the image and resize to a fixed size
            img_path = os.path.join(class_dir, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, winSize)

                # Compute the HOG descriptor for the image and store in the feature list
                features = hog.compute(img)
                X.append(features.ravel())

                # Extract the label from the filename and store in the label list
                label = class_name
                y.append(label)

# Convert the feature and label lists to numpy arrays
X = np.array(X)
y = np.array(y)
print("Feature array shape:", X.shape)
print("Label array shape:", y.shape)

# Perform 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    # Split the dataset into training and validation sets
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Train an SVM classifier on the training set
    clf = SVC(kernel='linear')
    clf.fit(X_train, y_train)

    # Predict the labels of the validation set
    y_pred = clf.predict(X_val)

    # Calculate the accuracy of the predictions
    accuracy = accuracy_score(y_val, y_pred)
    accuracies.append(accuracy)

    # Print the accuracy for this fold
    print("Fold %d accuracy: %f" % (fold+1, accuracy))

    # Calculate the accuracy of the predictions
    accuracy = accuracy_score(y_val, y_pred)
    accuracies.append(accuracy)

# Print the average accuracy and standard deviation
print("Average accuracy:", np.mean(accuracies))
print("Standard deviation:", np.std(accuracies))


Feature array shape: (1500, 1764)
Label array shape: (1500,)
Fold 1 accuracy: 0.973333
Fold 2 accuracy: 1.000000
Fold 3 accuracy: 0.993333
Fold 4 accuracy: 0.966667
Fold 5 accuracy: 0.973333
Fold 6 accuracy: 0.986667
Fold 7 accuracy: 0.980000
Fold 8 accuracy: 0.986667
Fold 9 accuracy: 0.986667
Fold 10 accuracy: 0.980000
Average accuracy: 0.9826666666666668
Standard deviation: 0.009521904571390453


In [52]:
from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score

# Create empty lists to store the true labels and predicted labels for all folds
y_true_all = []
y_pred_all = []

# Perform 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    # Split the dataset into training and validation sets
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Train an SVM classifier on the training set
    clf = SVC(kernel='linear')
    clf.fit(X_train, y_train)

    # Predict the labels of the validation set
    y_pred = clf.predict(X_val)

    # Calculate the accuracy of the predictions
    accuracy = accuracy_score(y_val, y_pred)
    accuracies.append(accuracy)

    # Add the true and predicted labels to the lists for all folds
    y_true_all.extend(y_val)
    y_pred_all.extend(y_pred)

    # Print the accuracy and confusion matrix for this fold
    print("Fold %d accuracy: %f" % (fold+1, accuracy))
    print("Confusion matrix:\n", confusion_matrix(y_val, y_pred))

# Calculate and print the overall performance metrics
print("Overall performance metrics:")
print("Accuracy:", accuracy_score(y_true_all, y_pred_all))
print("Confusion matrix:\n", confusion_matrix(y_true_all, y_pred_all))
print(classification_report(y_true_all, y_pred_all))
print("Cohen's Kappa:", cohen_kappa_score(y_true_all, y_pred_all))


Fold 1 accuracy: 0.973333
Confusion matrix:
 [[57  0  1]
 [ 0 40  1]
 [ 0  2 49]]
Fold 2 accuracy: 1.000000
Confusion matrix:
 [[46  0  0]
 [ 0 45  0]
 [ 0  0 59]]
Fold 3 accuracy: 0.993333
Confusion matrix:
 [[44  0  0]
 [ 0 54  1]
 [ 0  0 51]]
Fold 4 accuracy: 0.966667
Confusion matrix:
 [[50  0  0]
 [ 0 53  0]
 [ 0  5 42]]
Fold 5 accuracy: 0.973333
Confusion matrix:
 [[51  2  0]
 [ 0 57  0]
 [ 1  1 38]]
Fold 6 accuracy: 0.986667
Confusion matrix:
 [[51  0  0]
 [ 0 50  0]
 [ 1  1 47]]
Fold 7 accuracy: 0.980000
Confusion matrix:
 [[62  0  0]
 [ 0 46  2]
 [ 0  1 39]]
Fold 8 accuracy: 0.986667
Confusion matrix:
 [[46  0  0]
 [ 0 55  1]
 [ 0  1 47]]
Fold 9 accuracy: 0.986667
Confusion matrix:
 [[40  0  0]
 [ 1 43  1]
 [ 0  0 65]]
Fold 10 accuracy: 0.980000
Confusion matrix:
 [[49  0  1]
 [ 0 49  1]
 [ 0  1 49]]
Overall performance metrics:
Accuracy: 0.9826666666666667
Confusion matrix:
 [[496   2   2]
 [  1 492   7]
 [  2  12 486]]
              precision    recall  f1-score   support

 

In [54]:
from sklearn.metrics import confusion_matrix

# Get the confusion matrix
cm = confusion_matrix(y_val, y_pred, labels=classes)

# Calculate the true positives, false positives, true negatives, and false negatives
TP = np.diag(cm)
FP = cm.sum(axis=0) - TP
FN = cm.sum(axis=1) - TP
TN = cm.sum() - (TP + FP + FN)

# Calculate the specificity and sensitivity
specificity = TN / (TN + FP)
sensitivity = TP / (TP + FN)

print("Specificity:", specificity)
print("Sensitivity:", sensitivity)


Specificity: [1.   0.99 0.98]
Sensitivity: [0.98 0.98 0.98]


In [74]:


# Define the hyperparameter grid
param_grid = {'C': [0.1, 1, 10, 100]}

# Create a GridSearchCV object with a linear SVM classifier and the hyperparameter grid
clf = GridSearchCV(SVC(kernel='linear'), param_grid, cv=10)

# Fit the GridSearchCV object to the data
clf.fit(X, y)

# Print the best set of hyperparameters and the corresponding cross-validation score
print("Best C value:", clf.best_params_['C'])
print("Best cross-validation score:", clf.best_score_) 



Best C value: 0.1
Best cross-validation score: 0.9653333333333334


# KNeighborsClassifier

In [66]:
import cv2
import numpy as np
import os
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Define the image directory and HOG parameters
img_dir = "C:/WPy64-31110/notebooks/docs/Shoe vs Sandal vs Boot Image Dataset/"
classes = ['boot', 'sandal', 'shoe']
winSize = (64,64)
blockSize = (16,16)
blockStride = (8,8)
cellSize = (8,8)
nbins = 9

# Create the HOG descriptor object
hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins)

# Create empty lists to store the features and labels
X = []
y = []

# Loop through each image in the directory and extract features
for class_name in classes:
    class_dir = os.path.join(img_dir, class_name)
    for filename in os.listdir(class_dir):
        if filename.endswith((".jpg", ".jpeg")):
            # Load the image and resize to a fixed size
            img_path = os.path.join(class_dir, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, winSize)

                # Compute the HOG descriptor for the image and store in the feature list
                features = hog.compute(img)
                X.append(features.ravel())

                # Extract the label from the filename and store in the label list
                label = class_name
                y.append(label)

# Convert the feature and label lists to numpy arrays
X = np.array(X)
y = np.array(y)
print("Feature array shape:", X.shape)
print("Label array shape:", y.shape)

# Perform 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    # Split the dataset into training and validation sets
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Train a KNN classifier on the training set
    clf = KNeighborsClassifier(n_neighbors=5)
    clf.fit(X_train, y_train)

    # Predict the labels of the validation set
    y_pred = clf.predict(X_val)

    # Calculate the accuracy of the predictions
    accuracy = accuracy_score(y_val, y_pred)
    accuracies.append(accuracy)

    # Print the accuracy for this fold
    print("Fold %d accuracy: %f" % (fold+1, accuracy))

# Print the average accuracy and standard deviation
print("Average accuracy:", np.mean(accuracies))
print("Standard deviation:", np.std(accuracies))


Feature array shape: (1500, 1764)
Label array shape: (1500,)
Fold 1 accuracy: 0.973333
Fold 2 accuracy: 0.986667
Fold 3 accuracy: 0.966667
Fold 4 accuracy: 0.946667
Fold 5 accuracy: 0.953333
Fold 6 accuracy: 0.966667
Fold 7 accuracy: 0.940000
Fold 8 accuracy: 0.973333
Fold 9 accuracy: 0.953333
Fold 10 accuracy: 0.980000
Average accuracy: 0.9640000000000001
Standard deviation: 0.014360439485692024


In [71]:
from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score

# Create empty lists to store the true labels and predicted labels for all folds
y_true_all = []
y_pred_all = []

# Perform 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    # Split the dataset into training and validation sets
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Train a KNN classifier on the training set
    clf = KNeighborsClassifier(n_neighbors=5)
    clf.fit(X_train, y_train)

    # Predict the labels of the validation set
    y_pred = clf.predict(X_val)

    # Calculate the accuracy of the predictions
    accuracy = accuracy_score(y_val, y_pred)
    accuracies.append(accuracy)

    # Add the true and predicted labels to the lists for all folds
    y_true_all.extend(y_val)
    y_pred_all.extend(y_pred)

    # Print the accuracy and confusion matrix for this fold
    print("Fold %d accuracy: %f" % (fold+1, accuracy))
    cm = confusion_matrix(y_val, y_pred)
    print("Confusion matrix:\n", cm)

# Calculate and print the overall performance metrics
print("Overall performance metrics:")
print("Accuracy:", accuracy_score(y_true_all, y_pred_all))

# Get the confusion matrix for all data
cm_all = confusion_matrix(y_true_all, y_pred_all)
print("Confusion matrix:\n", cm_all)

from sklearn.metrics import confusion_matrix

# Get the confusion matrix
cm = confusion_matrix(y_val, y_pred, labels=classes)

# Calculate the true positives, false positives, true negatives, and false negatives
TP = np.diag(cm)
FP = cm.sum(axis=0) - TP
FN = cm.sum(axis=1) - TP
TN = cm.sum() - (TP + FP + FN)

# Calculate the specificity and sensitivity
specificity = TN / (TN + FP)
sensitivity = TP / (TP + FN)

print("Specificity:", specificity)
print("Sensitivity:", sensitivity)

# Calculate Cohen's Kappa
print("Cohen's Kappa:", cohen_kappa_score(y_true_all, y_pred_all))



Fold 1 accuracy: 0.973333
Confusion matrix:
 [[57  0  1]
 [ 0 39  2]
 [ 1  0 50]]
Fold 2 accuracy: 0.986667
Confusion matrix:
 [[46  0  0]
 [ 0 44  1]
 [ 1  0 58]]
Fold 3 accuracy: 0.966667
Confusion matrix:
 [[44  0  0]
 [ 0 54  1]
 [ 2  2 47]]
Fold 4 accuracy: 0.946667
Confusion matrix:
 [[50  0  0]
 [ 2 50  1]
 [ 2  3 42]]
Fold 5 accuracy: 0.953333
Confusion matrix:
 [[52  1  0]
 [ 3 53  1]
 [ 2  0 38]]
Fold 6 accuracy: 0.966667
Confusion matrix:
 [[51  0  0]
 [ 0 47  3]
 [ 2  0 47]]
Fold 7 accuracy: 0.940000
Confusion matrix:
 [[62  0  0]
 [ 2 44  2]
 [ 5  0 35]]
Fold 8 accuracy: 0.973333
Confusion matrix:
 [[46  0  0]
 [ 1 54  1]
 [ 1  1 46]]
Fold 9 accuracy: 0.953333
Confusion matrix:
 [[40  0  0]
 [ 2 42  1]
 [ 4  0 61]]
Fold 10 accuracy: 0.980000
Confusion matrix:
 [[50  0  0]
 [ 0 48  2]
 [ 0  1 49]]
Overall performance metrics:
Accuracy: 0.964
Confusion matrix:
 [[498   1   1]
 [ 10 475  15]
 [ 20   7 473]]
Specificity: [1.   0.99 0.98]
Sensitivity: [1.   0.96 0.98]
Cohen's K

In [72]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Define the range of hyperparameters to explore
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'p': [1, 2],
}

# Create the KNN classifier object
knn = KNeighborsClassifier()

# Create the grid search object
grid_search = GridSearchCV(knn, param_grid, cv=10)

# Fit the grid search object to the data
grid_search.fit(X, y)

# Print the best hyperparameters and the corresponding performance metrics
print("Best hyperparameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)
print("Test set score:", grid_search.score(X_test, y_test))


Best hyperparameters: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
Best cross-validation score: 0.9573333333333334
Test set score: 1.0


# CNN

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import os

# Define the image directory and CNN parameters
img_dir = "C:/WPy64-31110/notebooks/docs/Shoe vs Sandal vs Boot Image Dataset/"
classes = ['boot', 'sandal', 'shoe']
img_height = 64
img_width = 64
batch_size = 32

# Create a CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(classes))
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Create empty lists to store the features and labels
X = []
y = []

# Loop through each image in the directory and extract features
for class_name in classes:
    class_dir = os.path.join(img_dir, class_name)
    for filename in os.listdir(class_dir):
        if filename.endswith((".jpg", ".jpeg")):
            # Load the image and resize to a fixed size
            img_path = os.path.join(class_dir, filename)
            img = keras.preprocessing.image.load_img(
                img_path, target_size=(img_height, img_width)
            )
            img_array = keras.preprocessing.image.img_to_array(img)
            X.append(img_array)

            # Extract the label from the filename and store in the label list
            label = classes.index(class_name)
            y.append(label)

# Convert the feature and label lists to numpy arrays
X = np.array(X)
y = np.array(y)
print("Feature array shape:", X.shape)
print("Label array shape:", y.shape)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model on the training set
model.fit(X_train, y_train, epochs=10, batch_size=batch_size, verbose=0)

# Evaluate the model on the test set
_, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

# Print the test accuracy
print("Test accuracy:", test_accuracy)
# Perform 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    # Split the dataset into training and validation sets
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Train the model on the training set
    model.fit(X_train, y_train, epochs=10, batch_size=batch_size, verbose=0)

    # Evaluate the model on the validation set
    _, accuracy = model.evaluate(X_val, y_val, verbose=0)
    accuracies.append(accuracy)

    # Print the accuracy for this fold
    print("Fold %d accuracy: %f" % (fold+1, accuracy))

# Print the average accuracy and standard deviation
print("Average accuracy:", np.mean(accuracies))
print("Standard deviation:", np.std(accuracies))

Feature array shape: (1500, 64, 64, 3)
Label array shape: (1500,)
Test accuracy: 0.95333331823349
Fold 1 accuracy: 0.953333
Fold 2 accuracy: 0.993333
Fold 3 accuracy: 1.000000
Fold 4 accuracy: 1.000000
Fold 5 accuracy: 1.000000
Fold 6 accuracy: 1.000000
Fold 7 accuracy: 1.000000
Fold 8 accuracy: 1.000000
Fold 9 accuracy: 1.000000
Fold 10 accuracy: 1.000000
Average accuracy: 0.9946666657924652
Standard deviation: 0.013920413101189005


In [4]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold

# Initialize lists to store the true labels and predicted labels
true_labels = []
predicted_labels = []

# Perform 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    # Split the dataset into training and validation sets
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Train the model on the training set
    model.fit(X_train, y_train, epochs=10, batch_size=batch_size, verbose=0)

    # Evaluate the model on the validation set
    y_pred = model.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=1)
    true_labels.extend(y_val)
    predicted_labels.extend(y_pred_classes)


# Calculate the overall performance metrics
print("Overall classification report:\n")
print(classification_report(true_labels, predicted_labels, target_names=classes))




5/5 [==============================] - 0s 77ms/step
Overall classification report:

              precision    recall  f1-score   support

        boot       1.00      1.00      1.00       500
      sandal       1.00      1.00      1.00       500
        shoe       1.00      1.00      1.00       500

    accuracy                           1.00      1500
   macro avg       1.00      1.00      1.00      1500
weighted avg       1.00      1.00      1.00      1500



In [7]:
 from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, cohen_kappa_score
from sklearn.model_selection import KFold
# Use 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Create empty lists to store the true and predicted labels
true_labels = []
predicted_labels = []

# Loop through each fold
for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    # Split the dataset into training and validation sets
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Train the model on the training set
    model.fit(X_train, y_train, epochs=10, batch_size=batch_size, verbose=0)

    # Predict the labels for the validation set
    y_pred = model.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Append the true and predicted labels to the lists
    true_labels.extend(y_val)
    predicted_labels.extend(y_pred_classes)

# Calculate the overall accuracy, F1 score, and Cohen's Kappa
overall_accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average='macro')
kappa = cohen_kappa_score(true_labels, predicted_labels)

# Calculate the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Calculate the specificity and sensitivity for each class
num_classes = len(classes)
specificity = []
sensitivity = []
for i in range(num_classes):
    tp = cm[i][i]
    fp = np.sum(cm[:, i]) - tp
    fn = np.sum(cm[i, :]) - tp
    tn = np.sum(cm) - tp - fp - fn
    specificity.append(tn / (tn + fp))
    sensitivity.append(tp / (tp + fn))

# Print the results
print("Overall accuracy:", overall_accuracy)
print("Cohen's Kappa:", kappa)
print("Confusion matrix:")
print(cm)
print("Specificity:", specificity)
print("Sensitivity:", sensitivity)


5/5 [==============================] - 0s 37ms/step
Overall accuracy: 1.0
Cohen's Kappa: 1.0
Confusion matrix:
[[500   0   0]
 [  0 500   0]
 [  0   0 500]]
Specificity: [1.0, 1.0, 1.0]
Sensitivity: [1.0, 1.0, 1.0]


# FCNN

In [24]:
import cv2
import numpy as np
import os
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential

# Define the image directory and HOG parameters
img_dir = "C:/WPy64-31110/notebooks/docs/Shoe vs Sandal vs Boot Image Dataset/"
classes = ['boot', 'sandal', 'shoe']
winSize = (64,64)
blockSize = (16,16)
blockStride = (8,8)
cellSize = (8,8)
nbins = 9

# Create the HOG descriptor object
hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins)

# Create empty lists to store the features and labels
X = []
y = []

# Loop through each image in the directory and extract features
for class_name in classes:
    class_dir = os.path.join(img_dir, class_name)
    for filename in os.listdir(class_dir):
        if filename.endswith((".jpg", ".jpeg")):
            # Load the image and resize to a fixed size
            img_path = os.path.join(class_dir, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, winSize)

                # Compute the HOG descriptor for the image and store in the feature list
                features = hog.compute(img)
                X.append(features.ravel())

                # Extract the label from the filename and store in the label list
                label = class_name
                y.append(label)

# Convert the feature and label lists to numpy arrays
X = np.array(X)
y = np.array(y)

# Create a dictionary that maps class names to integer values
class_to_idx = {class_name: i for i, class_name in enumerate(classes)}

# Convert the labels to integer values
y = np.array([class_to_idx[label] for label in y])

print("Feature array shape:", X.shape)
print("Label array shape:", y.shape)

# Perform 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    # Split the dataset into training and validation sets
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Normalize pixel values to [0, 1]
    X_train = X_train / 255.0
    X_val = X_val / 255.0

    # Convert labels to one-hot encoding
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=3)

    # Define the model architecture
    model = Sequential([
        Flatten(input_shape=(X.shape[1],)),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(3, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model on the training set
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)


   #Evaluate the model on the validation set
    loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
    accuracies.append(accuracy)

    # Print the accuracy for this fold
    print("Fold %d accuracy: %f" % (fold+1, accuracy))

# Print the average accuracy and standard deviation
print("Average accuracy:", np.mean(accuracies))
print("Standard deviation:", np.std(accuracies))



Feature array shape: (1500, 1764)
Label array shape: (1500,)
Fold 1 accuracy: 0.873333
Fold 2 accuracy: 0.873333
Fold 3 accuracy: 0.880000
Fold 4 accuracy: 0.906667
Fold 5 accuracy: 0.893333
Fold 6 accuracy: 0.906667
Fold 7 accuracy: 0.906667
Fold 8 accuracy: 0.900000
Fold 9 accuracy: 0.886667
Fold 10 accuracy: 0.900000
Average accuracy: 0.8926666676998138
Standard deviation: 0.012806255810121007


In [31]:
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, cohen_kappa_score

# Create empty lists to store the true labels and predicted labels for all folds
y_true_all = []
y_pred_all = []

# Define the hyperparameters of the MLPClassifier
hidden_layer_sizes = (100, 50)
activation = 'relu'
solver = 'adam'
alpha = 0.0001
max_iter = 1000

# Perform 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    # Split the dataset into training and validation sets
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Train an FNN classifier on the training set
    clf = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation,
                        solver=solver, alpha=alpha, max_iter=max_iter)
    clf.fit(X_train, y_train)

    # Predict the labels of the validation set
    y_pred = clf.predict(X_val)

    # Calculate the accuracy of the predictions
    accuracy = accuracy_score(y_val, y_pred)
    accuracies.append(accuracy)

    # Add the true and predicted labels to the lists for all folds
    y_true_all.extend(y_val)
    y_pred_all.extend(y_pred)

    # Print the accuracy and confusion matrix for this fold
    print("Fold %d accuracy: %f" % (fold+1, accuracy))
    print("Confusion matrix:\n", confusion_matrix(y_val, y_pred))

# Calculate and print the overall performance metrics
print("Overall performance metrics:")
print("Accuracy:", accuracy_score(y_true_all, y_pred_all))
print("Confusion matrix:\n", confusion_matrix(y_true_all, y_pred_all))
print(classification_report(y_true_all, y_pred_all))
print("Cohen's Kappa:", cohen_kappa_score(y_true_all, y_pred_all))


Fold 1 accuracy: 0.973333
Confusion matrix:
 [[57  0  1]
 [ 0 39  2]
 [ 0  1 50]]
Fold 2 accuracy: 1.000000
Confusion matrix:
 [[46  0  0]
 [ 0 45  0]
 [ 0  0 59]]
Fold 3 accuracy: 0.980000
Confusion matrix:
 [[44  0  0]
 [ 0 53  2]
 [ 1  0 50]]
Fold 4 accuracy: 0.946667
Confusion matrix:
 [[50  0  0]
 [ 0 52  1]
 [ 0  7 40]]
Fold 5 accuracy: 0.960000
Confusion matrix:
 [[51  1  1]
 [ 1 54  2]
 [ 0  1 39]]
Fold 6 accuracy: 0.986667
Confusion matrix:
 [[51  0  0]
 [ 0 50  0]
 [ 1  1 47]]
Fold 7 accuracy: 0.973333
Confusion matrix:
 [[62  0  0]
 [ 0 46  2]
 [ 1  1 38]]
Fold 8 accuracy: 0.980000
Confusion matrix:
 [[46  0  0]
 [ 0 55  1]
 [ 1  1 46]]
Fold 9 accuracy: 0.993333
Confusion matrix:
 [[40  0  0]
 [ 1 44  0]
 [ 0  0 65]]
Fold 10 accuracy: 0.973333
Confusion matrix:
 [[49  0  1]
 [ 0 49  1]
 [ 0  2 48]]
Overall performance metrics:
Accuracy: 0.9766666666666667
Confusion matrix:
 [[496   1   3]
 [  2 487  11]
 [  4  14 482]]
              precision    recall  f1-score   support

 

In [44]:
# Create empty lists to store the true labels and predicted labels for all folds
y_true_all = []
y_pred_all = []

# Define the hyperparameters of the MLPClassifier
hidden_layer_sizes = (100, 50)
activation = 'relu'
solver = 'adam'
alpha = 0.0001
max_iter = 1000

# Perform 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    # Split the dataset into training and validation sets
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Train an FNN classifier on the training set
    clf = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation,
                        solver=solver, alpha=alpha, max_iter=max_iter)
    clf.fit(X_train, y_train)

    # Get the predictions for the validation data
    y_pred_fold = clf.predict(X_val)
    y_pred_all = np.concatenate([y_pred_all, y_pred_fold])
    
    # Get the true labels for the validation data
    y_true_all = np.concatenate([y_true_all, y_val])

# Get the unique labels in y_true_all
unique_labels = np.unique(y_true_all)

# Check if unique_labels is empty
if len(unique_labels) == 0:
    print("Error: no unique labels in y_true_all")
else:
    # Get the confusion matrix
    cm = confusion_matrix(y_true_all, y_pred_all, labels=unique_labels)

    # Calculate the true positives, false positives, true negatives, and false negatives
    TP = np.diag(cm)
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    # Calculate the specificity and sensitivity
    specificity = TN / (TN + FP)
    sensitivity = TP / (TP + FN)

    print("Specificity:", specificity)
    print("Sensitivity:", sensitivity)


Specificity: [0.994 0.985 0.986]
Sensitivity: [0.994 0.972 0.964]


In [5]:
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
y = iris.target

# Define the hyperparameter grid
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam', 'lbfgs'],
    'alpha': [0.0001, 0.001, 0.01],
    'max_iter': [1000, 2000, 3000]
}

# Create an MLPClassifier object
clf = MLPClassifier()

# Perform grid search with 10-fold cross-validation
grid_search = GridSearchCV(clf, param_grid, cv=10)
grid_search.fit(X, y)

# Print the best hyperparameters and the corresponding mean test score
print("Best hyperparameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)




C:\WPy64-31110\python-3.11.1.amd64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\WPy64-31110\python-3.11.1.amd64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\WPy64-31110\python-3.11.1.amd64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP:

C:\WPy64-31110\python-3.11.1.amd64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\WPy64-31110\python-3.11.1.amd64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\WPy64-31110\python-3.11.1.amd64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (sta

C:\WPy64-31110\python-3.11.1.amd64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\WPy64-31110\python-3.11.1.amd64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\WPy64-31110\python-3.11.1.amd64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (sta

C:\WPy64-31110\python-3.11.1.amd64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\WPy64-31110\python-3.11.1.amd64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\WPy64-31110\python-3.11.1.amd64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (sta

Best hyperparameters: {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'max_iter': 1000, 'solver': 'adam'}
Best score: 0.9866666666666667


# Random Forest 

In [12]:
import cv2
import numpy as np
import os
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, cohen_kappa_score
# Define the image directory and HOG parameters
img_dir = "C:/WPy64-31110/notebooks/docs/Shoe vs Sandal vs Boot Image Dataset/"
classes = ['boot', 'sandal', 'shoe']
winSize = (64,64)
blockSize = (16,16)
blockStride = (8,8)
cellSize = (8,8)
nbins = 9

# Create the HOG descriptor object
hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)

# Create empty lists to store the features and labels
X = []
y = []

# Loop through each image in the directory and extract features
for class_name in classes:
    class_dir = os.path.join(img_dir, class_name)
    for filename in os.listdir(class_dir):
        if filename.endswith((".jpg", ".jpeg")):
            # Load the image and resize to a fixed size
            img_path = os.path.join(class_dir, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, winSize)

                # Compute the HOG descriptor for the image and store in the feature list
                features = hog.compute(img)
                X.append(features.ravel())

                # Extract the label from the filename and store in the label list
                label = class_name
                y.append(label)

# Convert the feature and label lists to numpy arrays
X = np.array(X)
y = np.array(y)

# Create a dictionary that maps class names to integer values
class_to_idx = {class_name: i for i, class_name in enumerate(classes)}

# Convert the labels to integer values
y = np.array([class_to_idx[label] for label in y])

print("Feature array shape:", X.shape)
print("Label array shape:", y.shape)

# Perform 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    # Split the dataset into training and validation sets
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Create a random forest classifier and train it on the training set
    clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    clf.fit(X_train, y_train)

    # Predict the labels of the validation set
    y_pred = clf.predict(X_val)

    # Calculate the accuracy of the predictions
    accuracy = accuracy_score(y_val, y_pred)
    accuracies.append(accuracy)

    # Print the accuracy and confusion matrix for this fold
    print("Fold %d accuracy: %f" % (fold+1, accuracy))
    print("Confusion matrix:\n", confusion_matrix(y_val, y_pred))

# Print the average accuracy and standard deviation
print("Average accuracy:", np.mean(accuracies))
print("Standard deviation:", np.std(accuracies))
# Calculate and print the overall performance metrics
print("Overall performance metrics:")
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_val, y_pred))
print(classification_report(y_val, y_pred))
print("Cohen's Kappa:", cohen_kappa_score(y_val, y_pred))


Feature array shape: (1500, 1764)
Label array shape: (1500,)
Fold 1 accuracy: 0.960000
Confusion matrix:
 [[57  0  1]
 [ 0 40  1]
 [ 0  4 47]]
Fold 2 accuracy: 0.980000
Confusion matrix:
 [[46  0  0]
 [ 1 43  1]
 [ 1  0 58]]
Fold 3 accuracy: 0.973333
Confusion matrix:
 [[44  0  0]
 [ 1 53  1]
 [ 1  1 49]]
Fold 4 accuracy: 0.940000
Confusion matrix:
 [[50  0  0]
 [ 0 52  1]
 [ 1  7 39]]
Fold 5 accuracy: 0.960000
Confusion matrix:
 [[51  2  0]
 [ 0 55  2]
 [ 2  0 38]]
Fold 6 accuracy: 0.980000
Confusion matrix:
 [[51  0  0]
 [ 0 50  0]
 [ 1  2 46]]
Fold 7 accuracy: 0.953333
Confusion matrix:
 [[62  0  0]
 [ 1 46  1]
 [ 4  1 35]]
Fold 8 accuracy: 0.973333
Confusion matrix:
 [[46  0  0]
 [ 0 54  2]
 [ 1  1 46]]
Fold 9 accuracy: 0.960000
Confusion matrix:
 [[40  0  0]
 [ 0 44  1]
 [ 2  3 60]]
Fold 10 accuracy: 0.973333
Confusion matrix:
 [[49  0  1]
 [ 0 49  1]
 [ 0  2 48]]
Average accuracy: 0.9653333333333333
Standard deviation: 0.012220201853215592
Overall performance metrics:
Accuracy: 0

In [5]:


import cv2
import numpy as np
import os
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Define the image directory and HOG parameters
img_dir = "C:/WPy64-31110/notebooks/docs/Shoe vs Sandal vs Boot Image Dataset/"
classes = ['boot', 'sandal', 'shoe']
winSize = (64,64)
blockSize = (16,16)
blockStride = (8,8)
cellSize = (8,8)
nbins = 9

# Create the HOG descriptor object
hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins)

# Create empty lists to store the features and labels
X = []
y = []

# Loop through each image in the directory and extract features
for class_name in classes:
    class_dir = os.path.join(img_dir, class_name)
    for filename in os.listdir(class_dir):
        if filename.endswith((".jpg", ".jpeg")):
            # Load the image and resize to a fixed size
            img_path = os.path.join(class_dir, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, winSize)

                # Compute the HOG descriptor for the image and store in the feature list
                features = hog.compute(img)
                X.append(features.ravel())

                # Extract the label from the filename and store in the label list
                label = class_name
                y.append(label)

# Convert the feature and label lists to numpy arrays
X = np.array(X)
y = np.array(y)

# Create a dictionary that maps class names to integer values
class_to_idx = {class_name: i for i, class_name in enumerate(classes)}

# Convert the labels to integer values
y = np.array([class_to_idx[label] for label in y])

print("Feature array shape:", X.shape)
print("Label array shape:", y.shape)

# Perform 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    # Split the dataset into training and validation sets
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Define the model architecture
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # Train the model on the training set
    model.fit(X_train, y_train)

   #Evaluate the model on the validation set
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    accuracies.append(accuracy)

    # Print the accuracy for this fold
    print("Fold %d accuracy: %f" % (fold+1, accuracy))

# Print the average accuracy and standard deviation
print("Average accuracy:", np.mean(accuracies))
print("Standard deviation:", np.std(accuracies))

Feature array shape: (1500, 1764)
Label array shape: (1500,)
Fold 1 accuracy: 0.973333
Fold 2 accuracy: 0.986667
Fold 3 accuracy: 0.973333
Fold 4 accuracy: 0.933333
Fold 5 accuracy: 0.953333
Fold 6 accuracy: 0.980000
Fold 7 accuracy: 0.940000
Fold 8 accuracy: 0.966667
Fold 9 accuracy: 0.960000
Fold 10 accuracy: 0.966667
Average accuracy: 0.9633333333333335
Standard deviation: 0.01612451549659711


In [14]:
# Import the necessary libraries
from sklearn.metrics import confusion_matrix

# Perform 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Create empty lists to store true and predicted labels
true_labels = []
predicted_labels = []

# Loop through each fold in the cross-validation
for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    # Split the dataset into training and validation sets
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Define the model architecture
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # Train the model on the training set
    model.fit(X_train, y_train)

    # Evaluate the model on the validation set
    y_pred = model.predict(X_val)

    # Append the true and predicted labels to the corresponding lists
    true_labels.extend(y_val)
    predicted_labels.extend(y_pred)

# Compute the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Compute the sensitivity and specificity for each class
for i, class_name in enumerate(classes):
    tp = cm[i, i]
    fn = np.sum(cm[i, :]) - tp
    fp = np.sum(cm[:, i]) - tp
    tn = np.sum(cm) - tp - fn - fp

    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    print("Class:", class_name)
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)


Class: boot
Sensitivity: 0.992
Specificity: 0.982
Class: sandal
Sensitivity: 0.96
Specificity: 0.981
Class: shoe
Sensitivity: 0.938
Specificity: 0.982


In [17]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    "n_estimators": [50, 100, 150, 200, 250]
}

# Create a Random Forest classifier object
rf_classifier = RandomForestClassifier(random_state=42)

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=rf_classifier,
                           param_grid=param_grid,
                           cv=10,
                           n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(X, y)

# Print the best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)


Best hyperparameters: {'n_estimators': 200}
Best score: 0.9560000000000001
